In [25]:
import sys
import os

# Get the project directory (parent directory of the notebooks folder)
project_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_dir not in sys.path:
    sys.path.append(project_dir)

import load_data
import wave_detection
import annotate_raw
import epoch_creation
import event_cleaning
import extract_epochs
import filter
import first_vs_last
import my_plotting
import plot_average_waveforms
import plot_net_coverage
import spectrogram_plot
import statistical_analysis
import wave_classification
import wave_filtering
import mne
import yasa
import numpy as np
import pandas as pd
import statsmodels.api as sm
import ipywidgets
%matplotlib qt 

In [3]:
fname = "/Volumes/CSC-Ido/Analyze/101/N1/output/Strength_101_N1_forSW/annotated_raw_filtered.set"
raw = mne.io.read_raw_eeglab(fname,preload = True)

In [4]:
print(raw.info)

<Info | 8 non-empty values
 bads: []
 ch_names: E2, E3, E4, E5, E6, E7, E8, E9, E11, E14, E15, E16, E17, E18, ...
 chs: 198 EEG
 custom_ref_applied: False
 dig: 201 items (3 Cardinal, 198 EEG)
 highpass: 0.0 Hz
 lowpass: 50.0 Hz
 meas_date: unspecified
 nchan: 198
 projs: []
 sfreq: 100.0 Hz
>


In [20]:
events = mne.events_from_annotations(raw) # raw events

Used Annotations descriptions: ['End', 'NegPeak', 'PosPeak', 'Start']


In [ ]:

#sw = yasa.sw_detect(raw, hypno=hypno_up, include=(2, 3))
sw = yasa.sw_detect(raw, verbose=False, coupling=False);
df = sw.summary(); # general summary for each sw
df # Inspect the dataframe

In [22]:
# Define detection parameters
freq_sw = (0.5, 2)  
dur_neg = (0.3, 1.5)
dur_pos = (0.1, 1)
amp_neg = (30, 200)
amp_pos = (10, 150)
amp_ptp = (40, 350)
coupling = False
verbose = False

sw = yasa.sw_detect(raw, freq_sw=freq_sw, dur_neg=dur_neg, dur_pos=dur_pos, amp_neg=amp_neg, amp_pos=amp_pos, amp_ptp=amp_ptp, verbose=verbose, coupling=coupling)
sw_df = sw.summary()



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    2.4s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    5.0s


In [26]:
sw.plot_detection()

interactive(children=(IntSlider(value=0, description='Epoch:', layout=Layout(align_items='center', justify_con…

<function yasa.detection._DetectionResults.plot_detection.<locals>.update(epoch, amplitude, channel, win_size, filt)>

In [32]:
mask = sw.get_mask()
mask

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [39]:
data = raw.load_data()
data

sf = raw.info['sfreq']
sf




times = np.arange(data.size) / sf

AttributeError: 'RawEEGLAB' object has no attribute 'size'

In [40]:
data.info['']

<Info | 8 non-empty values
 bads: []
 ch_names: E2, E3, E4, E5, E6, E7, E8, E9, E11, E14, E15, E16, E17, E18, ...
 chs: 198 EEG
 custom_ref_applied: False
 dig: 201 items (3 Cardinal, 198 EEG)
 highpass: 0.0 Hz
 lowpass: 50.0 Hz
 meas_date: unspecified
 nchan: 198
 projs: []
 sfreq: 100.0 Hz
>

In [38]:
import seaborn as sns
import matplotlib.pyplot as plt

sw_highlight = data * mask
sw_highlight[sw_highlight == 0] = np.nan

plt.figure(figsize=(16, 4.5))

plt.plot(times, data, 'k')
plt.plot(times, sw_highlight, 'indianred')
plt.plot(events['NegPeak'], sw_highlight[(events['NegPeak'] * sf).astype(int)], 'bo', label='Negative peaks')
plt.plot(events['PosPeak'], sw_highlight[(events['PosPeak'] * sf).astype(int)], 'go', label='Positive peaks')
plt.plot(events['Start'], data[(events['Start'] * sf).astype(int)], 'ro', label='Start')

plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude (uV)')
plt.xlim([0, times[-1]])
plt.title('N3 sleep EEG data')
plt.legend()
sns.despine()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (198, 2) + inhomogeneous part.